# LAB VISUAL ETL TRANSFORMATION SCRIPT

In [ ]:
import os
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from data_task_helpers import something
from services.dte_tools.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task  # noqa: E402
import nest_asyncio
import asyncio
import sys
import time
from datetime import datetime

# Fix the Windows event loop warning BEFORE running the notebook
if sys.platform.startswith("win"):
    try:
        from asyncio import WindowsSelectorEventLoopPolicy
        asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())
    except ImportError:
        pass
    
nest_asyncio.apply()
params = {}

if __name__ == "__main__":
    environment = initialise_data_task(f"Tille Lab transformation", params=params)
    params["sleep_time"] = params.get("sleep_time", 1.5)

environment.log_message(f"Started Tille Lab Transformation {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
time.sleep(params["sleep_time"])

[2025-04-30T12:15:04.575971] Tille Lab transformation - INFO: Tille Lab transformation initialised running in active mode
Tille Lab transformation >>>: Tille Lab transformation initialised running in active mode
[2025-04-30T12:15:04.596498] Tille Lab transformation - INFO: Started Tille Lab Transformation 30/04/2025 12:15:04
Tille Lab transformation >>>: Started Tille Lab Transformation 30/04/2025 12:15:04


## Mysql Server Configuration

In [ ]:
labdashdb_config = {
    "USER": "root",
    "PASSWORD": "root",
    "HOST": "localhost",
    "DB": "labdashdb",
    "PORT": "3306",
}

labdashdb_url = f"mysql+pymysql://{labdashdb_config['USER']}:{labdashdb_config['PASSWORD']}@{labdashdb_config['HOST']}:{labdashdb_config['PORT']}/{labdashdb_config['DB']}"

try:
    labdashdb_conn = create_engine(labdashdb_url)
    environment.log_message(f"connected successfully to tille lab database: {labdashdb_config['DB']}")
    time.sleep(params["sleep_time"])
    
except Exception as e:
    environment.log_error(f"Error connecting to LabdashDB: {e}")
    sys.stdout.flush()
    sys.exit(1) 

[2025-04-30T12:15:06.129904] Tille Lab transformation - INFO: connected successfully to tille lab database: labdashdb
Tille Lab transformation >>>: connected successfully to tille lab database: labdashdb


## Mssql Server Configuration

In [32]:
def connect_to_database(database_name):
    return pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER=localhost;"
        f"DATABASE={database_name};"
        f"UID=sa;"
        f"PWD=root.R00T;"
        f"TrustServerCertificate=yes;",
        autocommit=True
    )

labvisualDB_conn = connect_to_database("master")
labvisualDB_cursor = labvisualDB_conn.cursor()

## Create Schemas

In [33]:
def create_schemas():
    try:
        schemas = ['source', 'derived', 'final', 'z', 'dbo']
        
        for schema in schemas:
            labvisualDB_cursor.execute(
                f"""
                    USE lab_visual_analysis;

                    IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = '{schema}')
                    BEGIN
                        EXEC('CREATE SCHEMA {schema}');
                    END
                """
            )

        labvisualDB_cursor.commit()
        time.sleep(params["sleep_time"])
    except Exception as e:
        environment.log_error(f"Error creating schemas: {e}")
        labvisualDB_cursor.rollback()
        sys.stdout.flush()
        sys.exit(1)

## Create Tables in the lab_visual_analysis database

In [34]:
def create_tables():
    table_creation_queries = [
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Facilities' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Facilities (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                HfrCode NVARCHAR(50),
                Name NVARCHAR(255),
                Region NVARCHAR(255),
                District NVARCHAR(255),
                Council NVARCHAR(255)
            );
        """,
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Device_Logs' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Device_Logs (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                DeviceName NVARCHAR(255),
                DeviceCode NVARCHAR(50),
                DateBrokenDown DATETIME2,
                DateReported DATETIME2,
                DateFixed DATETIME2,
                BreakDownReason NVARCHAR(255)
            );
        """,
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Commodity_Transactions' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Commodity_Transactions (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                CommodityName NVARCHAR(255),
                CommodityCode NVARCHAR(50),
                BatchNumber NVARCHAR(50),
                TransactionDate DATETIME2,
                ExpireDate DATETIME2,
                TransactionType NVARCHAR(50),
                TransactionQuantity INT
            );
        """,
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Sample' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Sample (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                sampletrackingid NVARCHAR(50),
                LabHfrCode NVARCHAR(50),
                HubHfrCode NVARCHAR(50),
                EntryModality NVARCHAR(50),
                SampleType NVARCHAR(255),
                TestName NVARCHAR(255),   
                SampleQualityStatus NVARCHAR(50),
                Results NVARCHAR(255),
                SampleRejectionReason NVARCHAR(255),
                DeviceName NVARCHAR(255),
                DeviceCode NVARCHAR(50),
                CollectionDate DATETIME2,
                ReceivedDate DATETIME2,
                TestDate DATETIME2,
                AuthorisedDate DATETIME2,
                DispatchDate DATETIME2
            );
        """
    ]

    for query in table_creation_queries:
        try:
            labvisualDB_cursor.execute(query)
        except Exception as e:
            environment.log_error(f"Error creating table: {e}")

    labvisualDB_cursor.commit()
    time.sleep(params["sleep_time"])


## Create lab_visual_analysis database

In [35]:
def create_lab_visual_analysis_database():
    global labvisualDB_conn, labvisualDB_cursor
    
    try:
        environment.log_message("Preparing Lab Visual Database")
        time.sleep(params["sleep_time"])
        
        labvisualDB_cursor.execute("SELECT name FROM sys.databases WHERE name = 'lab_visual_analysis'")
        labvisualDB_exists = labvisualDB_cursor.fetchone()

        if labvisualDB_exists:
            environment.log_message("Truncating LabVisual database tables")
            tables = ["tbl_Facilities", "tbl_Device_Logs", "tbl_Commodity_Transactions", "tbl_Sample"]

            for table in tables:
                try:
                    labvisualDB_cursor.execute(f"""
                        IF OBJECT_ID('source.{table}', 'U') IS NOT NULL
                            TRUNCATE TABLE source.{table}
                    """)

                except Exception as e:
                    environment.log_error(f"Error truncating table {table}: {e}")
                    labvisualDB_conn.rollback()
                    sys.stdout.flush()
                    sys.exit(1)

            labvisualDB_conn.commit()
            environment.log_message("Lab Visual Database preparation complete")
        else:
            labvisualDB_conn.execute("CREATE DATABASE lab_visual_analysis")
            labvisualDB_conn.commit()
            
            labvisualDB_conn = connect_to_database("lab_visual_analysis")
            labvisualDB_cursor = labvisualDB_conn.cursor()

            environment.log_message("Create Schemas")
            create_schemas()

            environment.log_message("Create Tables")
            create_tables()

            environment.log_message("Lab Visual Database preparation complete")
    except Exception as e:
        environment.log_error(f"Failed to prepare database: {e}")
        labvisualDB_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)

create_lab_visual_analysis_database()

[2025-04-30T12:15:07.713032] Tille Lab transformation - INFO: Preparing Lab Visual Database
Tille Lab transformation >>>: Preparing Lab Visual Database
[2025-04-30T12:15:09.260110] Tille Lab transformation - INFO: Truncating LabVisual database tables
Tille Lab transformation >>>: Truncating LabVisual database tables
[2025-04-30T12:15:09.276366] Tille Lab transformation - INFO: Lab Visual Database preparation complete
Tille Lab transformation >>>: Lab Visual Database preparation complete


## Load Facility Data

In [ ]:
def extract_and_insert_facility_data():
    health_facilities_masterlist = "./All_Operating_Health_Facilities_in_Tanzania-Lab-Visual-2021oct22.xlsx"
    
    if not os.path.exists(health_facilities_masterlist):
        environment.log_error(f"Excel file '{health_facilities_masterlist}' not found.")
        sys.stdout.flush()
        sys.exit(1)
    
    df_excel = pd.read_excel(health_facilities_masterlist)
    df_excel.rename(columns={"Facility Number": "HfrCode", "Facility Name": "Name"}, inplace=True)
    df_excel = df_excel[['HfrCode', 'Name', 'Region', 'District', 'Council']]
    df_excel['Region'] = df_excel['Region'].str.replace("Region", "", regex=True).str.strip()
    
    mysql_query = """
        SELECT
            mohswid AS HfrCode,
            facilityname AS Name,
            regionname AS Region,
            districtname AS District,
            council AS Council
        FROM hubfacilities
    """

    try:
        df_mysql = pd.read_sql(mysql_query, labdashdb_conn)
    except Exception as e:
        environment.log_error(f"Error fetching data from MySQL: {e}")
        sys.stdout.flush()
        sys.exit(1)
    
    # Merge the two datasets
    df_combined = pd.concat([df_excel, df_mysql])
    
    # Remove duplicates based on HfrCode
    df_combined.drop_duplicates(subset=['HfrCode'], keep='first', inplace=True)
    
    # Fetch existing HfrCodes to avoid primary key conflict
    try:
        existing_hfrcodes = pd.read_sql("SELECT HfrCode FROM source.tbl_Facilities", labvisualDB_conn)
        df_combined = df_combined[~df_combined['HfrCode'].isin(existing_hfrcodes['HfrCode'])]
    except Exception as e:
        environment.log_error(f"Error checking existing HfrCodes: {e}")
        sys.stdout.flush()
        sys.exit(1)

    insert_query = """
        INSERT INTO source.tbl_Facilities (HfrCode, Name, Region, District, Council)
        VALUES (?, ?, ?, ?, ?);
    """
    
    try:
        for _, row in df_combined.iterrows():
            labvisualDB_cursor.execute(insert_query, row['HfrCode'], row['Name'], row['Region'], row['District'], row['Council'])
            
        labvisualDB_conn.commit()
        if len(df_combined) > 0:
            environment.log_message(f"{len(df_combined)} rows inserted into tbl_Facilities.")
            time.sleep(params["sleep_time"])
    except Exception as e:
        environment.log_error(f"Error inserting data: {e}")
        labvisualDB_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)

    time.sleep(params["sleep_time"])

extract_and_insert_facility_data()

[2025-04-30T12:15:25.229053] Tille Lab transformation - INFO: 11401 rows inserted into tbl_Facilities.
Tille Lab transformation >>>: 11401 rows inserted into tbl_Facilities.


## Load Sample Data

In [37]:
def extract_and_insert_sample_data():
    query = """
        SELECT 
            DISTINCT trackingID as sampletrackingid, 
            facilityHfrID, 
            sampleType as SampleType, 
            testName as TestName, 
            sampleQuality as SampleQualityStatus, 
            rejectionReason as SampleRejectionReason, 
            sampleCollectionDate as CollectionDate, 
            dateReceivedLab as ReceivedDate, 
            results as Results, 
            testedDate as TestDate, 
            resultAuthorisedDate as AuthorisedDate, 
            resultAuthorisedDate as DispatchDate,
            testInstrument as DeviceName,
            NULL as DeviceCode,
            IF(SUBSTR(trackingID, 1, 4) = 'BC03', 'lab', 'hub') as EntryModality
        FROM tbl_labtests
        WHERE sampleCollectionDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
        OR dateSentLab >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
        OR dateReceivedLab >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
        OR registeredDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
        OR testedDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
        OR resultAuthorisedDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
        OR dateResultSentHub >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    """
    
    try:
        sample_data = pd.read_sql(query, labdashdb_conn)
        if sample_data.empty:
            environment.log_error("No sample data found.")
            sys.stdout.flush()
            sys.exit(1)

        def extract_hfr_code(row):
            if row['EntryModality'] == 'lab':
                return row['facilityHfrID'], None
            else:
                return None, row['facilityHfrID']

        sample_data[['LabHfrCode', 'HubHfrCode']] = sample_data.apply(extract_hfr_code, axis=1, result_type='expand')

        for _, row in sample_data.iterrows():
            insert_query = """
                INSERT INTO source.tbl_Sample (sampletrackingid, LabHfrCode, HubHfrCode, EntryModality, SampleType, 
                                            TestName, SampleQualityStatus, Results, SampleRejectionReason, DeviceName, 
                                            DeviceCode, CollectionDate, ReceivedDate, TestDate, AuthorisedDate, DispatchDate)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            
            labvisualDB_cursor.execute(insert_query, row['sampletrackingid'], row['LabHfrCode'], row['HubHfrCode'], 
                                row['EntryModality'], row['SampleType'], row['TestName'], row['SampleQualityStatus'], 
                                row['Results'], row['SampleRejectionReason'], row['DeviceName'], row['DeviceCode'], row['CollectionDate'], 
                                row['ReceivedDate'], row['TestDate'], row['AuthorisedDate'], row['DispatchDate'])
        
        labvisualDB_conn.commit()
        if len(sample_data) > 0:
            environment.log_message(f"{len(sample_data)} row inserted into tbl_Sample.")
            time.sleep(params["sleep_time"])

    except Exception as e:
        labvisualDB_conn.rollback()
        environment.log_error(f"Error inserting data: {e}")

extract_and_insert_sample_data()
time.sleep(params["sleep_time"])

[2025-04-30T12:15:37.686123] Tille Lab transformation - INFO: 8933 row inserted into tbl_Sample.
Tille Lab transformation >>>: 8933 row inserted into tbl_Sample.


## Load Device Logs Data

In [38]:
def extract_and_insert_device_log_data():
    query = """
        select
            deviceName as DeviceName,
            deviceCode as DeviceCode,
            dateBreakDown as DateBrokenDown,
            dateReported as DateReported,
            dateFixed as DateFixed,
            breakDownReason as BreakDownReason
        from
            instrumentlogs2
    """
    
    try:
        device_logs = pd.read_sql(query, labdashdb_conn)
        
        insert_query = """
        INSERT INTO source.tbl_Device_Logs (DeviceName, DeviceCode, DateBrokenDown, DateReported, DateFixed, BreakDownReason)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        
        for _, row in device_logs.iterrows():
            labvisualDB_cursor.execute(insert_query, row['DeviceName'], row['DeviceCode'], row['DateBrokenDown'], row['DateReported'], row['DateFixed'], row['BreakDownReason'])
        
        labvisualDB_conn.commit()
        if len(device_logs) > 0:
            environment.log_message(f"{len(device_logs)} rows inserted into tbl_Device_Logs.")
            time.sleep(params["sleep_time"])
    except Exception as e:
        environment.log_error(f"Error fetching or inserting device logs: {e}")
        labvisualDB_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)

extract_and_insert_device_log_data()
time.sleep(params["sleep_time"])

## Load Commodity Transaction Data

In [39]:
def extract_and_insert_commodity_transaction_data():
    query = """SELECT commodityName AS CommodityName, commodityCode AS CommodityCode, batchNo AS BatchNumber, transactionDate AS TransactionDate, 
                    expireDate AS ExpireDate, transactionType AS TransactionType, quantity AS TransactionQuantity FROM commoditytransactions"""
    
    try:
        commodity_transactions = pd.read_sql(query, labdashdb_conn)
        
        insert_query = """
            INSERT INTO source.tbl_Commodity_Transactions (CommodityName, CommodityCode, BatchNumber, TransactionDate, ExpireDate, TransactionType, TransactionQuantity)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        
        for _, row in commodity_transactions.iterrows():
            labvisualDB_cursor.execute(insert_query, row['CommodityName'], row['CommodityCode'], row['BatchNumber'], row['TransactionDate'],
                                row['ExpireDate'], row['TransactionType'], row['TransactionQuantity'])
        
        labvisualDB_conn.commit()
        if len(commodity_transactions) > 0:
            environment.log_message(f"{len(commodity_transactions)} rows inserted into tbl_Commodity_Transactions.")
            time.sleep(params["sleep_time"])
    
    except Exception as e:
        environment.log_error(f"Error fetching or inserting commodity transactions: {e}")
        labvisualDB_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)
        
extract_and_insert_commodity_transaction_data()
time.sleep(params["sleep_time"])

## Read the stored procedures file

In [40]:
def extract_stored_procedures_from_file():
    environment.log_message("Running ETL")
    time.sleep(params["sleep_time"])

    stored_procedures_file = "./create-stored-procedures.sql"
    if os.path.exists(stored_procedures_file):
        with open(stored_procedures_file, "r") as file:
            sql_commands = file.read()

        sql_batches = sql_commands.split("GO")
        environment.log_message("Add SP")
        
        for batch in sql_batches:
            batch = batch.strip()

            if not batch or batch.startswith("--") or batch.startswith("/*"):
                continue

            if batch:
                try:
                    labvisualDB_cursor.execute(batch)
                    labvisualDB_conn.commit()
                except Exception as e:
                    environment.log_error(f"Error loading batch: {e}")
                    labvisualDB_conn.rollback()
                    sys.stdout.flush()
                    sys.exit(1)
    else:
        environment.log_error(f"SQL file '{stored_procedures_file}' not found.")
        sys.stdout.flush()
        sys.exit(1)

extract_stored_procedures_from_file()

[2025-04-30T12:15:43.777219] Tille Lab transformation - INFO: Running ETL
Tille Lab transformation >>>: Running ETL
[2025-04-30T12:15:45.289648] Tille Lab transformation - INFO: Add SP
Tille Lab transformation >>>: Add SP


## Execute Data Transformation

In [41]:
def execute_stored_procedures():
    try:
        sp = f"EXEC dbo.sp_data_processing"
        labvisualDB_cursor.execute(sp)
        
        while labvisualDB_cursor.nextset():
            pass
        labvisualDB_conn.commit()

    except Exception as e:
        environment.log_error(f"Error executing stored procedure: {e}")
        labvisualDB_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)
        
    finally:
        if 'mssql_conn' in locals():
            labvisualDB_cursor.close()
            labvisualDB_conn.close()
            labdashdb_conn.dispose()

execute_stored_procedures()
environment.log_message("ETL complete")


[2025-04-30T12:19:48.866098] Tille Lab transformation - INFO: ETL complete
Tille Lab transformation >>>: ETL complete


In [42]:
time.sleep(params["sleep_time"])
environment.log_message(f"Completed Tille Lab Transformation {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

[2025-04-30T12:19:50.426371] Tille Lab transformation - INFO: Completed Tille Lab Transformation 30/04/2025 12:19:50
Tille Lab transformation >>>: Completed Tille Lab Transformation 30/04/2025 12:19:50
